In [1]:
import numpy as np
import pandas as pd

In [16]:
VARIANTS = ["Fuzz", "Distortion"]

### Helper

In [131]:
from pandas import DataFrame


def group_parts(BOM: DataFrame, variant: str):
  grouped = BOM.rename(columns={f"{variant} - Model/Value": "Model/Value"})

  variants = VARIANTS.copy()
  variants.remove(variant)

  for var in variants:
    grouped.drop(f"{var} - Model/Value", axis=1, inplace=True)

  grouped["Designator"] = grouped.groupby([f"Model/Value", "Package"])["Designator"].transform(lambda x: ", ".join(x))
  grouped.drop_duplicates(inplace=True)

  return grouped  

def group_parts_count(BOM: DataFrame, variant: str):
  grouped = BOM.rename(columns={f"{variant} - Model/Value": "Model/Value"})
  variants = VARIANTS.copy()
  variants.remove(variant)

  for var in variants:
    grouped.drop(f"{var} - Model/Value", axis=1, inplace=True)

  grouped.rename(columns={"Designator": "Quantity"}, inplace=True)

  grouped = grouped.groupby([f"Model/Value", "Package"]).count()

  #grouped = grouped.drop("-")

  return grouped

# Read BOM.xlsx

In [114]:
BOM = pd.read_excel("./BOM.xlsx")

# Assembly LUT 

## Fuzz

In [108]:
group_parts(BOM, "Fuzz")

,Designator,Package,Model/Value
0,U1,SOT-23-5,LMR62014XMF-NOPB
1,"U2, U3, U4",SOT-23-5,TL081H
4,U5,SOT-23-5,TLV767
5,"D1, D2, D3, D4",SOD-123,1N4148
9,D5,0805-2012,SD0805S020S1R0
10,D6,LED,Red LED
11,"J1, J2",Audio Jack,6.3mm Audio Jack
13,J3,GCT_USB4125-GF-A-0190_REVA2,USB4125-GF-A-0190
14,"JPF1, RD1",R_0805-2012,-
15,SW1,Foot Switch,6P DPDT Foot Switch


## Distortion

In [109]:
group_parts(BOM, "Distortion")

,Designator,Package,Model/Value
0,U1,SOT-23-5,LMR62014XMF-NOPB
1,"U2, U3, U4",SOT-23-5,TL081H
4,U5,SOT-23-5,TLV767
5,"D1, D2, D3, D4",SOD-123,1N4148
9,D5,0805-2012,SD0805S020S1R0
10,D6,LED,Red LED
11,"J1, J2",Audio Jack,6.3mm Audio Jack
13,J3,GCT_USB4125-GF-A-0190_REVA2,USB4125-GF-A-0190
14,"JPF1, R5",R_0805-2012,0Ω
15,SW1,Foot Switch,6P DPDT Foot Switch


# Order LUT

In [140]:
fuzz_quantity = 1
distortion_quantity = 0

fuzz_grouped = group_parts_count(BOM, "Fuzz")
fuzz_grouped["Quantity"] = fuzz_grouped["Quantity"] * fuzz_quantity
distortion_grouped = group_parts_count(BOM, "Distortion")
distortion_grouped["Quantity"] = distortion_grouped["Quantity"] * distortion_quantity

pd.concat([fuzz_grouped, distortion_grouped]).groupby(["Model/Value", "Package"]).sum().sort_values("Package")

,,Quantity
Model/Value,Package,
SD0805S020S1R0,0805-2012,1
10uH,Abracon ASPI 3012S,1
6.3mm Audio Jack,Audio Jack,2
-,C_0805-2012,2
100nF,C_0805-2012,3
470nF,C_0805-2012,2
4.7uF,C_0805-2012,2
10uF,C_0805-2012,3
330pF,C_0805-2012,1


In [141]:
fuzz_quantity = 0
distortion_quantity = 1

fuzz_grouped = group_parts_count(BOM, "Fuzz")
fuzz_grouped["Quantity"] = fuzz_grouped["Quantity"] * fuzz_quantity
distortion_grouped = group_parts_count(BOM, "Distortion")
distortion_grouped["Quantity"] = distortion_grouped["Quantity"] * distortion_quantity

pd.concat([fuzz_grouped, distortion_grouped]).groupby(["Model/Value", "Package"]).sum().sort_values("Package")

,,Quantity
Model/Value,Package,
SD0805S020S1R0,0805-2012,1
10uH,Abracon ASPI 3012S,1
6.3mm Audio Jack,Audio Jack,2
-,C_0805-2012,2
100nF,C_0805-2012,3
470nF,C_0805-2012,2
4.7uF,C_0805-2012,2
10uF,C_0805-2012,3
330pF,C_0805-2012,1
